# Model Tuning / Model Doğrulama

In [10]:
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn import model_selection
import matplotlib.pyplot as plt

In [11]:
hit = pd.read_csv("Hitters.csv")

df = hit.copy()

df = df.dropna()

dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

y = df["Salary"]

X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")

X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

lasso_model = Lasso(alpha = 0.1).fit(X_train, y_train)

y_pred_train = lasso_model.predict(X_train)

y_pred_test = lasso_model.predict(X_test)

C:\Users\Sefa3\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8249174.751388096, tolerance: 3898.686956380658
  positive)


In [12]:
np.sqrt(mean_squared_error(y_train, y_pred_train))

289.3655052843862

In [13]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

356.7545270148768

In [14]:
lasso_cv_model = LassoCV(alphas = None, cv = 10, max_iter = 10000, normalize = True)

In [15]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
        max_iter=10000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [16]:
lasso_cv_model.alpha_

0.39406126432470073

In [17]:
lasso_tuned = Lasso(alpha = lasso_cv_model.alpha_)

In [18]:
lasso_tuned.fit(X_train, y_train)

C:\Users\Sefa3\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8252823.943523985, tolerance: 3898.686956380658
  positive)


Lasso(alpha=0.39406126432470073, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

## final modelimiz : test seti için;

In [19]:
y_pred_test = lasso_tuned.predict(X_test)

In [20]:
np.sqrt(mean_squared_error(y_test, y_pred_test))  # ridge regresyonuna göre modelimiz daha başarılı.

356.5226376958367

In [22]:
r2_score(y_test, y_pred_test)

0.41282875506652295

## final modelimiz : eğitim seti için;

In [23]:
y_pred_train = lasso_tuned.predict(X_train)

In [26]:
np.sqrt(mean_squared_error(y_train, y_pred_train))  # ridge regresyonuna göre modelimiz daha başarılı.

289.3772303515828

In [25]:
r2_score(y_train, y_pred_train)

0.5768673163703463

-----

Makine Öğrenmesi algoritmaları cross validation yöntemi ile doğrulanıp,

bu doğrulanan cross validation yöntemi sayesinde tespit edilen optimum parametre değerleri ile

oluşturulan final modellerinin, TEST hataları üzerinden değerlendirme hatasını ele alınır.

ve bu hata üzerinden tahminler, çıkarımlar yapılır.